In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15102MB, multi_processor_count=40)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np
torch.manual_seed(42)


device = torch.device('cuda:0')

In [5]:
print(dir(torchvision.datasets))

['CIFAR10', 'CIFAR100', 'CLEVRClassification', 'CREStereo', 'Caltech101', 'Caltech256', 'CarlaStereo', 'CelebA', 'Cityscapes', 'CocoCaptions', 'CocoDetection', 'Country211', 'DTD', 'DatasetFolder', 'EMNIST', 'ETH3DStereo', 'EuroSAT', 'FER2013', 'FGVCAircraft', 'FakeData', 'FallingThingsStereo', 'FashionMNIST', 'Flickr30k', 'Flickr8k', 'Flowers102', 'FlyingChairs', 'FlyingThings3D', 'Food101', 'GTSRB', 'HD1K', 'HMDB51', 'INaturalist', 'ImageFolder', 'ImageNet', 'Imagenette', 'InStereo2k', 'KMNIST', 'Kinetics', 'Kitti', 'Kitti2012Stereo', 'Kitti2015Stereo', 'KittiFlow', 'LFWPairs', 'LFWPeople', 'LSUN', 'LSUNClass', 'MNIST', 'Middlebury2014Stereo', 'MovingMNIST', 'Omniglot', 'OxfordIIITPet', 'PCAM', 'PhotoTour', 'Places365', 'QMNIST', 'RenderedSST2', 'SBDataset', 'SBU', 'SEMEION', 'STL10', 'SUN397', 'SVHN', 'SceneFlowStereo', 'Sintel', 'SintelStereo', 'StanfordCars', 'UCF101', 'USPS', 'VOCDetection', 'VOCSegmentation', 'VisionDataset', 'WIDERFace', '__all__', '__builtins__', '__cached__',

In [6]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download = True, transform = transform)
testset = torchvision.datasets.CIFAR10(root="./data", train=False, download = True, transform = transform)

100%|██████████| 170498071/170498071 [00:04<00:00, 40730952.22it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
sample,_ = trainset[0]

trainset.data.shape, torch.Size(sample.size())

((50000, 32, 32, 3), torch.Size([3, 32, 32]))

In [8]:
trainloader = DataLoader(trainset, batch_size=100, shuffle = True)
testloader = DataLoader(testset, batch_size=100, shuffle = False)

In [9]:
class CNN_arch(nn.Module):
    def __init__(self):
        torch.manual_seed(42)
        super(CNN_arch, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=6,kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2= nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)

        # reason for 16*5*5 is in def forward
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 32*32*3 -> input size
        x_conv1 = self.pool(F.relu(self.conv1(x)))
        # 28*28*6 out from conv1
        # 14*14*6 out from pool after conv1
        x_conv2 = self.pool(F.relu(self.conv2(x_conv1)))
        # 10*10*16 out from conv2
        # 5*5*16 out from pool after conv2
        x_flat = x_conv2.view(-1, 16*5*5)
        x_fc_1 = F.relu(self.fc1(x_flat))
        x_fc_2 = F.relu(self.fc2(x_fc_1))
        out = self.fc3(x_fc_2)
        return out



In [14]:
model = CNN_arch().to(device) # move model to gpu
# model = CNN_arch()
print(model)
Loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

CNN_arch(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [15]:
# Count the total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters:", total_params)

Total number of parameters: 62006


In [ ]:
# Run a forward pass through the model using the training data to calculate the initial loss value
# initial_loss = 0.0
# total_samples = 0
# with torch.no_grad():
#     for inputs, labels in trainloader:
#         # inputs, labels = inputs.to(device), labels.to(device)
#         outputs = model(inputs)
#         loss = Loss(outputs, labels)
#         initial_loss += loss.item() * labels.size(0)
#         total_samples += labels.size(0)


# initial_loss /= total_samples
# print("Initial Loss:", initial_loss)

Initial Loss: 2.305333071231842


In [16]:
epochs = 2
for epoch in range(epochs):
    running_loss = 0.0
    for i,data in enumerate(trainloader,0):
        inputs, labels = data
        optimizer.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = Loss(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

'''
[1,   100] loss: 2.297
[1,   200] loss: 2.114
[1,   300] loss: 1.849
[1,   400] loss: 1.701
[1,   500] loss: 1.598
[2,   100] loss: 1.533
[2,   200] loss: 1.493
[2,   300] loss: 1.445
[2,   400] loss: 1.432
[2,   500] loss: 1.381 --> without GPU
'''

[1,   100] loss: 2.297
[1,   200] loss: 2.113
[1,   300] loss: 1.848
[1,   400] loss: 1.699
[1,   500] loss: 1.598
[2,   100] loss: 1.532
[2,   200] loss: 1.493
[2,   300] loss: 1.447
[2,   400] loss: 1.429
[2,   500] loss: 1.375


'\n[1,   100] loss: 2.297\n[1,   200] loss: 2.114\n[1,   300] loss: 1.849\n[1,   400] loss: 1.701\n[1,   500] loss: 1.598\n[2,   100] loss: 1.533\n[2,   200] loss: 1.493\n[2,   300] loss: 1.445\n[2,   400] loss: 1.432\n[2,   500] loss: 1.381 --> without GPU\n'

In [17]:
final_loss = 0.0
total_baches = 0.0

for i, data in enumerate(trainloader,0):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)
    loss = Loss(outputs, labels)
    final_loss += loss
    total_baches+=1
final_loss/=total_baches

final_loss.item()

1.3410794734954834

In [19]:
correct = 0
total = 0

# Disable gradient tracking during testing
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate accuracy
accuracy = correct / total
print('Accuracy of the network on the test dataset:', accuracy)

Accuracy of the network on the test dataset: 0.5084
